In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from typing import *

Although we have EVs in summary file, we need to get EV ratios from the raw results for accurate bootstrap summaries

In [6]:
protein='1fme'
batch_num = 1
raw_path = f'../{protein}/raw_batch{batch_num}.h5'
summary_path = f'../{protein}/summary_batch{batch_num}.h5'
timescales = pd.DataFrame(pd.read_hdf(raw_path, key='timescales'))

timescales.reset_index(inplace=True)
timescales.rename(mapper={0: 'value'}, axis=1, inplace=True)

In [7]:
eigenvalues = timescales.copy(deep=True)
eigenvalues['value'] = timescales.apply(lambda x: ev(x), axis=1)

In [11]:
eigenvalues.sort_values(by=['hp_ix', 'bs_ix', 'lag', 'process'], inplace=True)
# eigenvalues['ratio'] = eigenvalues.groupby(['hp_ix', 'bs_ix', 'lag', 'process'])['value'].diff()

In [27]:
eigenvalues['log_ev'] = np.log(eigenvalues['value'])

In [29]:
eigenvalues['ev_ratio'] = np.exp(eigenvalues.groupby(['hp_ix', 'bs_ix', 'lag'])['log_ev'].diff(periods=-1))

In [36]:
print(eigenvalues.shape[0])
eigenvalues.dropna(inplace=True)
eigenvalues.shape[0]

5812692


5663235

In [38]:
ev_summary = eigenvalues.groupby(['hp_ix', 'lag', 'process'])['ev_ratio'].agg(
            median=lambda x: np.nanquantile(x, 0.5),
            lb=lambda x: np.nanquantile(x, 0.025),
            ub=lambda x: np.nanquantile(x, 0.975),
            count=lambda x: x.shape[0]-x.isna().sum()
)
ev_summary.reset_index(inplace=True)

In [39]:
ev_summary.to_hdf(summary_path, key='eigenvalue_ratio')